Reproduce the dataset:

**Legit**: domain -> screenshots

**Phishing**: domain -> screenshots (same as above for our purposes)

**New Phishing**: domain -> different resolution screenshots -> prove that diff resolution will cause performance drop

In [ ]:
#!pip install selenium
#!apt install chromium-chromedriver
#print("---------- done ----------")

Utility for resize image if not using standard resolution

In [ ]:
from PIL import Image  

def resize_img(src_fn):
    # Opens a image in RGB mode  
    im = Image.open(src_fn) 

    #print("original size:{}".format(im.size))
    newsize = (1024, 768)
    im1 = im.resize(newsize)
    #print("new size:{}".format(im1.size))
    im1.save(src_fn)


Read top 50 domains

In [ ]:

urlbydomain = {}
counter=0
with open('url_log.csv',encoding="utf-8") as fp1:
    
    for row in fp1:
        idx=row.find(",")
        
        if(idx==-1):
            #print(row)
            continue
        #print(row)
        counter+=1
        label=row[:idx]
        url=row[idx+1:].strip().strip("\n")
        #print("{}-->{}".format(label,url))
        if(label not in urlbydomain):
            urlbydomain[label]=[]
        urlbydomain[label].append(url)
            
        
#print(len(urlbydomain["youtube.com"]))
for label in urlbydomain:
    urllist = urlbydomain[label]
    if(len(urllist)<100):
        #print(label)
        continue
print(counter)

In [ ]:
#dummy cell to exec shell command

#!rm -rf /content/dataset_new_phishing_noresize/*
# !find dataset_new_phishing_noresize/ -name *.png | wc -l

In [ ]:
from selenium import webdriver
import bs4
import os
import random
import hashlib

o = webdriver.ChromeOptions()
o.add_argument("--headless")
o.add_argument("--no-sandbox")
o.add_argument("--disable-dev-shm-usage")
#o.add_argument("user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 14_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 Safari 604.1")
driver = webdriver.Chrome('chromedriver',options=o)
driver.set_window_position(0, 0)
w=1366
h=768
driver.set_window_size(w, h)
dataset_name = "dataset_new_phishing_{}_{}_unseen".format(w,h)

agent = driver.execute_script("return navigator.userAgent")
print(agent)

In [ ]:





#crawl the domain, then find k sub link, collect those screenshots
random.seed(128)

started=True
if(os.path.exists(dataset_name) == False):
        os.mkdir(dataset_name)
for label in urlbydomain:
    # if(label=="gravatar.com"):    #for continued search
    #     started=True
    # if(started==False):
    #     continue
    urllist = list(urlbydomain[label])
    if(label in ["qq.com"]):#too slow,skip
        continue
    if(len(urllist)<100):
        #print(label)
        continue
    print("label:{}".format(label))
    #continue
    
    
    
    random.shuffle(urllist) #shuffle it (but with a fixed random seed)
    urllist[0]="https://{}".format(label)#make sure mother included
    basepath=os.path.join(dataset_name,label)
    counter=0
    for url in urllist:

        try:
            driver.get(url)
        
            
            if(counter>50):#crawl first 100
                break
            if(os.path.exists(basepath) == False):
                os.mkdir(basepath)
            #print("{}:{}".format(counter,url))
            fn = hash_object = hashlib.md5(url.encode()).hexdigest()[:10]
            filepath = os.path.join(basepath,fn+".png")
            #print(path)
            driver.save_screenshot(filepath)
            #resize_img(filepath)
            counter+=1
        except Exception as e:
            print(e)
            continue
    print("domain {} done".format(label))
print("all done.")
    #now saved all domains


In [ ]:
#compress everything for download
#!rm /content/dataset_benign.tar.gz
#!tar -czf dataset_benign.tar.gz dataset/
#!rm /content/dataset_phishing.tar.gz
#!tar -czf dataset_new_phishing_noresize.tar.gz dataset_new_phishing_noresize/